In [ ]:
import pandas as pd
import geopandas as gpd
from siuba import *

import datetime as dt
import os

# Example 4-day aggregation using speedmap data

* Export 4 days in April to support MTC Plan Bay Area 2050
* First, run speedmap pipeline for any days not already ran (can filter to subset of operators/districts if desired by using additional scripts and editing Makefile, see d4 example)

In [ ]:
dates = [f'2023-04-{day}' for day in range(10, 14)]

In [ ]:
dates

In [ ]:
progress_all_dates = []
for date in dates:
    df = pd.read_parquet(f'./_rt_progress_{date}.parquet')
    df = df >> filter(_.caltrans_district == '04 - Oakland')
    progress_all_dates += [df]
four_days = pd.concat(progress_all_dates)

Basic checks on organizations ran

In [ ]:
# four_days >> count(_.organization_name, _.status, _.organization_itp_id)

In [ ]:
# four_days >> filter(_.status == 'already_ran')

In [ ]:
am_filter = {'period': 'am', 'filter_args': {'start_time': '06:00', 'end_time': '09:00'}}
mid_filter = {'period': 'mid', 'filter_args': {'start_time': '10:00', 'end_time': '14:00'}}
pm_filter = {'period': 'pm', 'filter_args': {'start_time': '15:00', 'end_time': '19:00'}}
all_filters = [am_filter, mid_filter, pm_filter]

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
from rt_analysis import signal_tools

In [ ]:
signal_tools.concatenate_speedmap_segments?

In [ ]:
pbar = tqdm()

## loop over dates, using `signal_tools` to aggregate all operators in each time period

In [ ]:
# for date in dates:
#     progress_df = pd.read_parquet(f'./_rt_progress_{date}.parquet')
#     progress_df = progress_df >> filter(_.caltrans_district == '04 - Oakland') >> filter(_.status == 'map_confirmed')
#     gdfs = {}
#     for time_filter in all_filters:
#         gdfs[time_filter['period']] = signal_tools.conacatenate_speedmap_segments(progress_df = progress_df, pbar=pbar, filter_args=time_filter['filter_args'])
#     for period in gdfs.keys():
#         gdfs[period].to_parquet(f'_{date}_{period}.parquet')

## concatenate/aggregate 4 day period

* start by aggregating metrics for each time of day period across all dates/operators
* then prefix am/mid/pm observations and concat

In [ ]:
def read_all_dates(period):
    gdfs = []
    for date in dates:
        gdfs += [gpd.read_parquet(f'_{date}_{period}.parquet')]
    return pd.concat(gdfs)

In [ ]:
all_am = read_all_dates('am')

In [ ]:
all_pm = read_all_dates('pm')

In [ ]:
all_mid = read_all_dates('mid')

In [ ]:
group_cols = ['geometry', 'shape_id', 'stop_sequence',
       'route_id', 'route_short_name', 'direction_id', 'gtfs_dataset_key', 'organization_name',
             'miles_from_last']

In [ ]:
prefix_cols = ['p50_mph', 'p20_mph',
       'p80_mph', 'fast_slow_ratio', 'trips_per_hour',
       'time_formatted', 'system_p50_median', 'n_weekdays']

In [ ]:
def aggregate_prefix(period_concat_df, period):

    counts = (period_concat_df >> count(_.shape_id, _.stop_sequence, _.gtfs_dataset_key, _.organization_name)
          >> select(_.shape_id, _.stop_sequence, _.gtfs_dataset_key, _.organization_name, _.n_weekdays == _.n))
    all_period_agged = period_concat_df.groupby(group_cols).median().reset_index()
    all_period_agged = all_period_agged >> inner_join(_, counts, on = ['shape_id', 'stop_sequence', 'gtfs_dataset_key', 'organization_name'])
    prefix = period + '_'
    prefixed = [prefix + col for col in prefix_cols]
    rename_dict = dict(zip(prefix_cols, prefixed))
    all_period_agged = all_period_agged.rename(columns=rename_dict)
    
    return all_period_agged

In [ ]:
am_prefixed = aggregate_prefix(all_am, 'am')

In [ ]:
pm_prefixed = aggregate_prefix(all_pm, 'pm')

In [ ]:
mid_prefixed = aggregate_prefix(all_mid, 'mid')

In [ ]:
joined = (am_prefixed >> full_join(_, mid_prefixed, on=group_cols)
          >> full_join(_, pm_prefixed, on=group_cols)
         
         )

In [ ]:
joined.columns

## re-add stop id and stop name

* can't group on these since not present for virtual segments!

In [ ]:
df = pd.concat([all_am, all_mid, all_pm]) >> distinct(_.shape_id, _.stop_sequence, _.gtfs_dataset_key,
                                                     _.stop_id, _.stop_name)

In [ ]:
joined = joined >> left_join(_, df, on=['shape_id', 'stop_sequence', 'gtfs_dataset_key'])

In [ ]:
joined >> count(_.stop_id, _.stop_name) >> arrange(-_.n)

In [ ]:
joined = gpd.GeoDataFrame(joined)

In [ ]:
joined.crs

In [ ]:
# (joined >> filter(_.stop_id.isna())).explore()

In [ ]:
joined.to_parquet('d4_apr10_apr13_express_fixed2.parquet')

In [ ]:
gdf = gpd.read_parquet('./d4_apr10_apr13_express_fixed.parquet')

In [ ]:
(gdf >> filter(_.route_short_name.isin(['Lynx']))).explore()

In [ ]:
(gdf >> filter(_.route_short_name.isin(['FCX']))).explore()

In [ ]:
(gdf >> filter(_.route_short_name.isin(['F', 'E', 'NL']))).explore()

In [ ]:
(gdf >> filter(_.organization_name.str.contains('Golden'))).route_short_name.unique()

In [ ]:
joined.columns